# Big project

## Submission

Submit solutions to https://forms.office.com/e/WexY7YraJb.

1.   Upload code in .ipynb file
2.   Upload a csv containing three columns: 
*    "ID": the ID of the instance (1, 2, 3, ...)
*    "OBJ": the objective function value obtained
*    "TIME": the execution time in seconds.

## Evaluation

1.   Gap w.r.t. optimal solutions
2.   Runtimes. Must be under 10 minutes for every instance. Execution times will be re-examined on a random basis.

## Deadline

12/01/2022 23:59 CET

## Other

*   4 lab points just if you deliver something that works
*   10 points based on the quality of method
*   NO pre-coded libraries, 
*   NO genetic algorithms 
*   NO neural networks
*   groups of max 3 students




## Biogas plants location

An association of $n$ farmers wants to open $p$ plants to produce energy from biogas. 
Each plant will be opened at a farm of a member of the association and will be powered with corn chopping purchased from the farm itself or from other neighboring farms.

Each farm $i$ can provide at most $c_i$ tons of corn chopping, with a percentage of dry matter $a_i$. As you may know, dry matter is the key component of corn chopping used for biogas production. In order to maintain the quality of produced energy, each plant must burn a mixture of corn chopping with a percentage of dry matter between $k_{min}$ and $k_{max}$. 

At most one plant can be located in each farm, and every farm can sell its corn chopping to one and only one plant.

Each farm $i$ is located at coordinates $x_i$ and $y_i$, representing respectively its latitude and longitude, and the cost of moving corn chopping from a farm $i$ to a farm $j$ is proportional to the euclidean distance between the two farms (it does not depend on the actual quantity moved, since the trucks used for this transportations are sufficiently big). 

Under such conditions, every plant produces $Q$ kWh of energy per ton of corn chopping burned. The energy produced by each plant will be fed into the national electricity system, at a unitary price of $b$ (€/kWh). Moreover, due to state regulations, each plant must not produce more than $M$ kWh of energy.

You must locate $p$ plants among the available farms and assign the farms that will supply each plant, with the goal of maximizing the total revenues of the association.

### Sets
*   $I$ = set of farms

### Parameters
*   $n$ = number of farms   
*   $p$ = number of plants to locate
*   $b$ = revenue per unit of energy (€/kWh)
*   $M$ = max energy production (kWh)
*   $Q$ = energy produced by a ton of corn chopping (kWh/t)
*   $k_{min} (k_{max})$ = min (max) percentage of dry matter for fermentation
*   $a_i$ = percentage of dry matter in chopping from farm $i \in I$
*   $c_i$ = tons of corn chopping available for each $i \in I$ (t)
*   $x_i, y_i$ = coordinates of farm $i \in I$

##CONSIDERAZIONI DI LERRY
* dovremmo fare una funzione ausiliaria per il calcolo della distanza tra farm e poi metterle (magari con pandas) in una matrice che rappresenta le distanze tra una farm e l'altra (ovviamente la distanza tra se stessa sarà nulla).
dovrebbe venire una matrice quadrata, in questo modo abbiamo una matrice di distanze che possiamo utilizzare per localizzare dove mettere le farm
* Una farm che ospita una plant può comprare da più farm o solo da una designata? Per come lo intesa io una plant Pi è associata ad una o più farm Nj(quindi dovremmo specificare che la cardinalità di P è minore o uguale a N)
* Analizzare l'input che ci viene dato potrebbe essere un ottimo punto di partenza per iniziare a ragionare, inoltre ogni plant è uguale come produzione, è possibile trasformare la max energy production M in un quanto corn chop serve
facendo M/Q, quello è il massimo da usare come constraint.
* da quello che ho capito dobbiamo organizzare i dati per creare poi un modello mip da utilizzare, penso sia questa la difficoltà grossa. I dati a noi forniti sono molto più grezzi in fondo...



In [2]:
# Clean files if present
!rm instance_1.json
!rm instance_1.txt

# Download instance_1 (both instance and solution) directly from Github
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_1.json
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_1.txt
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_2.json
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_2.txt
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_3.json
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_3.txt
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_4.json
!wget https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_4.txt

--2022-12-26 13:54:36--  https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 572 [text/plain]
Saving to: ‘instance_1.json’

instance_1.json     100%[===================>]     572  --.-KB/s    in 0s      

2022-12-26 13:54:36 (25.5 MB/s) - ‘instance_1.json’ saved [572/572]

--2022-12-26 13:54:36--  https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


In [3]:
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json

import mip 
import math

from datetime import datetime

# Reads a .json instance and returns it in a dictionary
def load_instance(filename):
  with open(filename, 'r') as f:
    data = json.load(f)
  return data

# Reads a .txt result and returns it
def load_result(filename):
  with open(filename, 'r') as f:
      result = f.read()
  return float(result)

def solve(inst):
  I = range(inst["n"])
  M = inst["M"]
  Q = inst["Q"]
  a = inst["a"]
  b = inst["b"]
  c = inst["c"]
  kmax = inst["kmax"]
  kmin = inst["kmin"]
  p = inst["p"]

  d = [[0 if (i == j) else math.sqrt( (inst["points"][i][0] - inst["points"][j][0])**2 + (inst["points"][i][1] - inst["points"][j][1])**2  ) for i in I] for j in I]

  m = mip.Model()

  alpha = [m.add_var(var_type=mip.BINARY) for i in I]
  beta = [[m.add_var() for i in I] for j in I] #i plant, j farm
  gamma = [[m.add_var(var_type=mip.BINARY) for i in I] for j in I]

  m.objective = mip.maximize( mip.xsum( (b * Q * beta[i][j] - d[i][j] * gamma[i][j])  for i in I for j in I) )
  #mip.xsum( b * Q * mip.xsum( beta[i][j] for j in I) - mip.xsum(d[i][j] * gamma[i][j] for j in I) for i in I)

  m.add_constr( mip.xsum( alpha[i] for i in I) == p)
  for i in I:
    for j in I:
      m.add_constr( beta[i][j] <= c[j] * gamma[i][j]) #constraint che gamma è 1 se b[i][j] > 0

  for i in I:
    m.add_constr(mip.xsum( beta[i][j] * a[j] for j in I) <= kmax * mip.xsum( beta[i][j] for j in I) )
    m.add_constr(mip.xsum( beta[i][j] * a[j] for j in I) >= kmin * mip.xsum( beta[i][j] for j in I) )
    m.add_constr( Q * mip.xsum( beta[i][j] for j in I ) <= M)
    m.add_constr( mip.xsum(gamma[i][j] for j in I) <= (M/Q) * alpha[i]) #farm can sell to plant only if plant is active, M/Q come costante (non può prendere più corn chops di così)

  for j in I: 
    m.add_constr( mip.xsum(gamma[i][j] for i in I) <= 1)
    m.add_constr( mip.xsum(beta[i][j] for i in I) <= c[j])
  
  status = m.optimize()
  print(status)
  return m.objective_value

for k in {1, 2, 3, 4}:
  print("INSTANCE #" + str(k))
  inst = load_instance("instance_" + str(k) + ".json")
  res = load_result("instance_" + str(k) + ".txt")
  print("start time: " + str(datetime.now()))
  obj = solve(inst)
  print("end time: " + str(datetime.now()))
  gap = 100 * (obj - res) / res

  print("result #" + str(k) + ": {}".format(obj))
  print("expected #" + str(k) + ": {}".format(res))
  print("gap #" + str(k) + ": {}".format(gap))


In [4]:
import mip 
import math


inst = load_instance("instance_1.json")

I = range(inst["n"])

print("inst")
print(inst)
#for i in I: 
#    print(inst["points"][i][0], inst["points"][i][1], sep=" ")
 #   print()


I = range(inst["n"])
M = inst["M"]
Q = inst["Q"]
a = inst["a"]
b = inst["b"]
c = inst["c"]
kmax = inst["kmax"]
kmin = inst["kmin"]
p = inst["p"]

d = [[0 if (i == j) else math.sqrt( (inst["points"][i][0] - inst["points"][j][0])**2 + (inst["points"][i][1] - inst["points"][j][1])**2  ) for i in I] for j in I]

print("d")
for i in I: 
  for j in I:
    print(d[i][j], end=" " )
  print("\n")


m = mip.Model()

alpha = [m.add_var(var_type=mip.BINARY) for i in I]
beta = [[m.add_var() for i in I] for j in I] #i plant, j farm
gamma = [[m.add_var(var_type=mip.BINARY) for i in I] for j in I]

m.objective = mip.maximize( mip.xsum( (b * Q * beta[i][j] - d[i][j] * gamma[i][j])  for i in I for j in I) )
#mip.xsum( b * Q * mip.xsum( beta[i][j] for j in I) - mip.xsum(d[i][j] * gamma[i][j] for j in I) for i in I)

m.add_constr( mip.xsum( alpha[i] for i in I) == p)
for i in I:
  for j in I:
    m.add_constr( beta[i][j] <= c[j] * gamma[i][j]) #constraint che gamma è 1 se b[i][j] > 0

for i in I:
  m.add_constr(mip.xsum( beta[i][j] * a[j] for j in I) <= kmax * mip.xsum( beta[i][j] for j in I) )
  m.add_constr(mip.xsum( beta[i][j] * a[j] for j in I) >= kmin * mip.xsum( beta[i][j] for j in I) )
  m.add_constr( Q * mip.xsum( beta[i][j] for j in I ) <= M)
  m.add_constr( mip.xsum(gamma[i][j] for j in I) <= (M/Q) * alpha[i]) #farm can sell to plant only if plant is active, M/Q come costante (non può prendere più corn chops di così)

for j in I: 
  m.add_constr( mip.xsum(gamma[i][j] for i in I) <= 1)
  m.add_constr( mip.xsum(beta[i][j] for i in I) <= c[j])

result = m.optimize()

print(result)
print(m.objective_value)

inst
{'M': 2211000, 'Q': 477, 'a': [0.33629281460324734, 0.36037630265352005, 0.23591024519268208, 0.5476134135031488, 0.5694452530438001], 'b': 0.4468190156768048, 'c': [1967, 1250, 452, 1866, 756], 'kmax': 0.4340371592993856, 'kmin': 0.15419754067090946, 'n': 5, 'p': 2, 'points': [[26.202467501558168, 15.868397215446562], [27.812651949435974, 45.931688721456666], [32.100054052016716, 51.839282059753714], [26.194292556514466, 97.60852848774336], [73.28145526904824, 11.527422668314946]]}
d
0 30.10638121946121 36.4511465334433 81.74013168109065 47.27869656864838 

30.10638121946121 0 7.299416130068584 51.70217457085462 57.01811640148193 

36.4511465334433 7.299416130068584 0 46.148693778132 57.62771740918822 

81.74013168109065 51.70217457085462 46.148693778132 0 98.11808024728299 

47.27869656864838 57.01811640148193 57.62771740918822 98.11808024728299 0 

OptimizationStatus.OPTIMAL
1324477.6736137536
